# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

import torch
from collections import deque
import datetime;

import json
config = json.load(open('config.json'))
train = True

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [2]:
env = UnityEnvironment(file_name="Tennis_Windows_x86_64/Tennis.exe",no_graphics=train, seed = config['seed']['environment'])

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=train)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1] *2
print(state_size)
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
print('The state for the first agent looks like:', states[1])

Number of agents: 2
Size of each action: 2
48
There are 2 agents. Each observes a state with length: 48
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.64731312 -1.5
 -0.          0.          7.99828482  6.         -0.          0.        ]
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -7.15059757 -1.5
  0.          0.         -7.99828482  6.          0.          0.        ]


In [5]:
#variables to store values for plotting
scores = []
scores_window = deque(maxlen=100)
avg_scores = []
solved_episode = -1
game_length_window =deque(maxlen=100)
game_length = []
avg_game_length = []
network_loss = []

# hyperparameters
GAMMA = config['GAMMA']            # discount factor
TAU = config['TAU']              # for soft update of target parameters
LR_ACTOR = config['LR_ACTOR']        # learning rate of the actor 
LR_CRITIC = config['LR_CRITIC']        # learning rate of the critic
WEIGHT_DECAY = config['WEIGHT_DECAY']       # L2 weight decay

noise=config['noise_factor']
noise_decay=config['noise_decay']

def concatStates(states):
    concated_states = []
    concated_states.append(np.reshape(states,(1,states[1].size*2)))
    concated_states.append(np.reshape(states[::-1],(1,states[1].size*2)))
    return torch.FloatTensor(concated_states)
        

In [6]:
from maddpg import MADDPG
from er import ReplayBuffer

def seeding(seed=1):
    np.random.seed(seed)
    torch.manual_seed(seed)    
    
buffer = ReplayBuffer(action_size, config['BUFFER_SIZE'], config['BATCH_SIZE'], config['seed']['agent'])

maddpg = MADDPG(config, state_size, action_size, num_agents, tau=TAU)
print(maddpg.maddpg_agent)
print(json.dumps(config, indent=2))
#print(agents[0].actor_local)
#print(agents[0].critic_local)
device = 'cpu'

[<ddpg.DDPGAgent object at 0x000002BB9FDC8470>, <ddpg.DDPGAgent object at 0x000002BB9FDF4208>]
{
  "train_per_episode": 2,
  "TAU": 0.01,
  "LR_CRITIC": 0.001,
  "LR_ACTOR": 0.001,
  "GAMMA": 0.99,
  "BUFFER_SIZE": 50000,
  "BATCH_SIZE": 256,
  "WEIGHT_DECAY": 0.0001,
  "noise": {
    "sigma": 1.0,
    "mu": 0.0,
    "theta": 0.15
  },
  "noise_factor": 0.6,
  "noise_decay": 0.998,
  "random": {
    "seed": 42,
    "count": 0
  },
  "seed": {
    "environment": 42,
    "agent": 6
  },
  "network": {
    "hidden": 256
  }
}


In [7]:
#seed replay buffer with random actions
np.random.seed(config['random']['seed'])
count_positiv_random_rewards =0
count_all_actions = 0
for i in range(1, config['random']['count']):                                      # play game for 5 episodes
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    ep_scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    while True:
        actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
        actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        next_states = env_info.vector_observations         # get next state (for each agent)
        rewards = env_info.rewards                         # get reward (for each agent)
        dones = env_info.local_done                        # see if episode finished
        ep_scores += env_info.rewards                         # update the score (for each agent)
        states = next_states     # roll over states to next time step
        
        if np.any(dones):                                  # exit loop if episode finished
            break
        
        states_for_replay = concatStates(env_info.vector_observations)
        next_states_for_replay = concatStates(next_states)
        [buffer.add(states_for_replay[i], actions[i], sum(rewards), next_states_for_replay[i], dones[i]) for i in range(num_agents)]
        #print(states_for_replay[0], actions[0], sum(rewards), next_states_for_replay[0], dones[0])
        count_all_actions+= 1
        if (np.max(ep_scores) > 0):
            count_positiv_random_rewards += 1

print('Random Seed with {:d}/{:d} hits'.format(count_positiv_random_rewards,count_all_actions))

Random Seed with 0/0 hits


In [ ]:
#runtime config
episodes = 2000
expected_score = 0.5
seeding(config['seed']['agent'])
actor_loss = []
critic_loss = []
for i in range(100):
    scores_window.append(0)
for episode in range(1, episodes+1):                                    # play game for n episodes
    start = datetime.datetime.now()
    ep_scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    count = 0
    
    env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    states = concatStates(env_info.vector_observations)
    
    maddpg.resetNoise()
    while True:        
        count += 1
        #print('states', states.shape)
        actions = maddpg.act(states, noise=noise)                       # select an action (for each agent)
        #print('actions', actions)
        env_info = env.step(actions)[brain_name]           # send all actions to tne environment
        #next_states = env_info.vector_observations         # get next state (for each agent)        
        next_states = concatStates(env_info.vector_observations)
        
        rewards = env_info.rewards                         # get reward (for each agent)
        #print('reward', rewards)
        dones = env_info.local_done                        # see if episode finished
        #print('dones', dones)
        ep_scores += env_info.rewards                         # update the score (for each agent)
        
        #print('s',states.shape)
        #print('ss',states[0].shape)
        [buffer.add(states[i], actions[i], sum(rewards), next_states[i], dones[i]) for i in range(num_agents)]
        
       
        
        states = next_states                               # roll over states to next time step
        if np.any(dones):                                  # exit loop if episode finished
            break
    #print(ep_scores)
    for i in range(config['train_per_episode']):         
        if (config['BATCH_SIZE'] <= len(buffer)):
            aloss=[]
            closs=[]
            for i in range(num_agents):
                loss = maddpg.update(buffer.sample(), i, GAMMA)
                aloss.append(loss[0])   
                closs.append(loss[1])
            actor_loss.append(aloss)
            critic_loss.append(closs)
        #maddpg.update_targets()
        
    noise *= noise_decay
    
    took = datetime.datetime.now() -start    
    ep_score = np.max(ep_scores)
    scores.append(ep_score)
    scores_window.append(ep_score)
    avg_score = np.mean(scores_window)
    avg_scores.append(avg_score)    
    game_length.append(count)
    game_length_window.append(count)
    avg_game_length.append(np.mean(game_length_window))
    if (ep_score > 0 or episode%10 == 0):
        print('Score (max over agents) from episode\t{}:\t{:.2f}\tAverage Score:\t{:.4f}\ttime:\t{:s}\tactions:\t{}\tNoise:\t{:.5f}\t{:d}'.format(episode, ep_score, avg_score, str(took), count, noise, len(buffer)))
    if (solved_episode < 0 and avg_score >= expected_score):
        print('Solved in episode {:d}'.format(episode))
        solved_episode = episode
        [torch.save(actor.state_dict(), 'solved_{:d}_actor{i}_local.pth'.format(i, episode))for i, actor in enumerate(maddpg.get_actors())]
        [torch.save(critic.state_dict(), 'solved_{:d}_critic{i}_local.pth'.format(i, episode))for i, critic in enumerate(maddpg.get_critics())]        

print('Solved in episode {}'.format(solved_episode))

[torch.save(actor.state_dict(), 'end_actor{}_local.pth'.format(i))for i, actor in enumerate(maddpg.get_actors())]
[torch.save(critic.state_dict(), 'end_critic{}_local.pth'.format(i))for i, critic in enumerate(maddpg.get_critics())] 

Score (max over agents) from episode	10:	0.00	Average Score:	0.0000	time:	0:00:00.168571	actions:	14	Noise:	0.58811	284
Score (max over agents) from episode	20:	0.00	Average Score:	0.0000	time:	0:00:00.185014	actions:	15	Noise:	0.57645	568
Score (max over agents) from episode	30:	0.00	Average Score:	0.0000	time:	0:00:00.206952	actions:	15	Noise:	0.56502	852
Score (max over agents) from episode	37:	0.09	Average Score:	0.0009	time:	0:00:00.261312	actions:	31	Noise:	0.55716	1084
Score (max over agents) from episode	40:	0.00	Average Score:	0.0009	time:	0:00:00.183528	actions:	14	Noise:	0.55383	1168
Score (max over agents) from episode	50:	0.00	Average Score:	0.0009	time:	0:00:00.206844	actions:	14	Noise:	0.54285	1452
Score (max over agents) from episode	56:	0.10	Average Score:	0.0019	time:	0:00:00.298340	actions:	31	Noise:	0.53637	1656
Score (max over agents) from episode	60:	0.00	Average Score:	0.0019	time:	0:00:00.202458	actions:	14	Noise:	0.53209	1770
Score (max over agents) from episod

When finished, you can close the environment.

In [ ]:
env.close()

In [ ]:
import matplotlib.pyplot as plt

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.plot(np.arange(len(avg_scores)), avg_scores)
#plt.plot(np.arange(len(scores)), expected_score*np.ones(len(scores)))
plt.xlabel('Episode number')
plt.ylabel('Score')
plt.savefig('graph_trained_{:d}_episodes_{:d}_{:d}.png'.format(len(scores),config['seed']['environment'],config['seed']['agent']), format='png')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(actor_loss)), actor_loss)
plt.xlabel('learn step')
plt.ylabel('actor Loss')
plt.savefig('graph_actor_loss_{:d}_episodes_{:d}_{:d}.png'.format(len(scores),config['seed']['environment'],config['seed']['agent']), format='png')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(critic_loss)), critic_loss)
plt.xlabel('learn step')
plt.ylabel('critic Loss')
plt.savefig('graph_critic_loss_{:d}_episodes_{:d}_{:d}.png'.format(len(scores),config['seed']['environment'],config['seed']['agent']), format='png')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(game_length)), game_length)
plt.plot(np.arange(len(avg_game_length)), avg_game_length)
plt.xlabel('Episode number')
plt.ylabel('Turns')
plt.savefig('graph_gamelength_{:d}_episodes_{:d}_{:d}.png'.format(len(scores),config['seed']['environment'],config['seed']['agent']), format='png')
plt.show()